In [1]:
import pandas as pd

from gazetteer_loc import GazetteerAdmLoc as GZTR

In [2]:
# Prepared dataframe
f_in = '/home/admin/text_processing/anotated_data/ner_data/sample_angin_topan_gold_dataset_ner_w_tokens.xlsx'
df_ner = pd.read_excel(f_in)
df_data = df_ner[df_ner.raw_ner.notnull()].copy()

In [3]:
df_out = df_ner[['id']].copy()
df_out['adm1'] = ''
df_out['adm2'] = ''
df_out['adm3'] = ''

In [4]:
# Initilize gazetteer
gztr = GZTR()

In [5]:
def search_in_gazetteer(sr_locs):
    dict_adm1 = {}
    dict_adm2 = {}
    dict_adm3 = {}
    for index, item in sr_locs.iteritems():
        loc_id, loc_str = gztr.search_gazetteer_by_name(4, index, sr_locs, 1)
        if None != loc_id and loc_id not in dict_adm1:
            dict_adm1[loc_id] = loc_str

        loc_id, loc_str = gztr.search_gazetteer_by_name(4, index, sr_locs, 2)
        if None != loc_id and loc_id not in dict_adm2:
            dict_adm2[loc_id] = loc_str

        loc_id, loc_str = gztr.search_gazetteer_by_name(4, index, sr_locs, 3)
        if None != loc_id and loc_id not in dict_adm3:
            dict_adm3[loc_id] = loc_str
            
    return dict_adm1, dict_adm2, dict_adm3

In [6]:
class ADMLoc:
    dict_adm3 = {}
    dict_adm2 = {}
    dict_adm1 = {}

In [7]:
def consolidate_adm3(adm_loc):
    adm3_str = ''
    tmp_dict_adm3 = {}
    dict_adm2 = adm_loc.dict_adm2
    dict_adm3 = adm_loc.dict_adm3
        
    adm3_keys = [ ".".join(k.split('.')[:-1]) for k in list(dict_adm3.keys()) ]
    adm3_prefix_keys = [ k for k in adm3_keys if k in dict_adm2 ]
    for key in adm3_prefix_keys:
        key = key + "."
        tmp_dict = { k : v for k, v in dict_adm3.items() if k.startswith(key) }
        tmp_dict_adm3.update(tmp_dict)
    
    adm_loc.dict_adm3 = tmp_dict_adm3
    return adm_loc
    

In [8]:
def consolidate_adm2(adm_loc):
    dict_adm2 = adm_loc.dict_adm2
    dict_adm3 = adm_loc.dict_adm3
    
    # if empty the check adm3
    if 0 >= len(list(dict_adm2.keys())):
        adm3_keys = [ ".".join(k.split('.')[:-1]) for k in list(dict_adm3.keys()) ]
        new_adm2_keys = [ k for k in adm3_keys if k not in dict_adm2.keys() ]
        sr_id = pd.Series(new_adm2_keys)
        
        for index, item in sr_id.iteritems():
            loc_id, loc_str = gztr.search_gazetteer_by_id(1, index, sr_id, 2)
            if None != loc_id and None != loc_str:
                dict_adm2[loc_id] = loc_str
    
    adm_loc.dict_adm2 = dict_adm2
    return adm_loc

In [9]:
def consolidate_adm1(adm_loc):
    dict_adm1 = adm_loc.dict_adm1
    dict_adm2 = adm_loc.dict_adm2
    
    # Refer to adm2 list
    adm2_keys = [ ".".join(k.split('.')[:-1]) for k in list(dict_adm2.keys()) ]
    new_adm1_keys = [ k for k in adm2_keys if k not in dict_adm1 ]
    sr_id = pd.Series(new_adm1_keys)
    
    for index, item in sr_id.iteritems():
        loc_id, loc_str = gztr.search_gazetteer_by_id(1, index, sr_id, 1)
        if None != loc_id and None != loc_str:
            dict_adm1[loc_id] = loc_str
    
    
    adm_loc.dict_adm1 = dict_adm1
    return adm_loc

In [10]:
tmp_df = df_data[df_data.id == 'angin_topan_0013']
tmp_df

,id,raw_ner,exec_time
12,angin_topan_0013,"Jakarta, Selatan",2.408049


In [11]:
for index, row in df_data[df_data.id == 'angin_topan_0013'].iterrows():
    adm_loc = ADMLoc()
    row_id = row['id']
    sr_locs = pd.Series([ l.strip() for l in row['raw_ner'].split(',')])
    adm_loc.dict_adm1, adm_loc.dict_adm2, adm_loc.dict_adm3 =\
        search_in_gazetteer(sr_locs)
    
    adm_loc = consolidate_adm2(adm_loc)
    adm_loc = consolidate_adm3(adm_loc)
    adm_loc = consolidate_adm1(adm_loc)
    
    adm1_str = ', '.join(list(adm_loc.dict_adm1.values()))
    adm2_str = ', '.join(list(adm_loc.dict_adm2.values()))
    adm3_str = ', '.join(list(adm_loc.dict_adm3.values()))
    
    df_out.loc[df_out['id'] == row_id, 'adm1'] = adm1_str         
    df_out.loc[df_out['id'] == row_id, 'adm2'] = adm2_str
    df_out.loc[df_out['id'] == row_id, 'adm3'] = adm3_str

/home/admin/venv-jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
